# Practice activity: Applying PEFT

## Introduction
Parameter-efficient fine-tuning (PEFT) is a technique that reduces the computational cost and memory requirements of fine-tuning large pretrained models. Instead of updating all of the model’s parameters, PEFT focuses on fine-tuning a smaller subset of the model’s parameters while keeping most of the model's original weights frozen. This approach allows for faster training times and lower memory usage, making fine-tuning more feasible for large-scale models. In this reading, we'll explore the key steps for applying PEFT to a pretrained model and the benefits of using this technique.

**By the end of this activity, you will be able to:**
- Understand the concept of PEFT and its advantages.
- Identify key parameters for fine-tuning and apply the PEFT technique to a pretrained model.
- Implement a fine-tuning process with reduced computational cost and memory usage.
- Evaluate and optimize the performance of the fine-tuned model using PEFT.

## Why use PEFT?
Traditional fine-tuning methods require updating all of the model’s parameters, which can be computationally expensive, especially for large models such as GPT-3, BERT, or T5. PEFT offers several benefits:

- **Reduced computational cost:** By only fine-tuning a subset of the model’s parameters, you can significantly reduce the amount of computational resources needed.
- **Lower memory requirements:** PEFT uses less memory since only a few parameters are updated, making it easier to fine-tune on smaller graphics processing units (GPUs) or machines with limited resources.
- **Faster training times:** With fewer parameters to update, the training process is much faster, allowing for quicker iterations and experiments.

## Step-by-step process for applying PEFT
This reading will guide you through the following steps:

**Step 1:** Prepare your data and identify the subset of parameters for fine-tuning

**Step 2:** Set up fine-tuning with PEFT

**Step 3:** Monitor and evaluate performance

**Step 4:** Optimize PEFT for your task

### Step 1: Prepare your data and identify the subset of parameters for fine-tuning
Before beginning the fine-tuning process, it is essential to ensure that your dataset is properly prepared. You should be working with a task-specific dataset (e.g., sentiment analysis, text classification) that aligns with the pretrained model you’ll be using. Preprocess the data, ensuring it’s tokenized and ready for input into the model. For this activity, we’ll assume you’re working with a classification task, but this process can also be adapted for other tasks.

#### Instructions for preparing your data
- Ensure that your dataset is cleaned and preprocessed.
- Tokenize the data using a tokenizer compatible with the pretrained model (e.g., BERT tokenizer for a BERT model).
- Split your dataset into training, validation, and test sets.

Once your data is ready, the next step is identifying which parameters to fine-tune. In PEFT, we often fine-tune the parameters in the task-specific heads, which are the layers responsible for generating predictions based on the task. For models like BERT, the task-specific heads are the final few layers, usually the classification head.

#### Locate the task-specific heads
In a BERT-based model, task-specific heads typically refer to the layers at the end of the model used for tasks such as classification, where the model generates outputs based on the input data.

You can inspect the model architecture to find these heads and determine which layers are responsible for your task.

#### Approach
To implement PEFT, you will freeze most of the model’s parameters, allowing only the parameters in the task-specific heads (final layers) to be updated. This strategy minimizes computational cost while allowing the model to adapt to your specific task.

#### Customize fine-tuning
You can also choose to fine-tune multiple layers if your task requires more adaptation. For example, you might fine-tune the last two or three layers instead of just the final classification head. This gives you more flexibility in training while still taking advantage of the efficiency of PEFT.

In [8]:
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv("data/tweet_emotion_intensity.csv")


# Split dataset into training (70%), validation (15%), and test (15%)
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")




Training set size: 2772
Validation set size: 594
Test set size: 594


In [9]:
# Load pre-trained BERT model
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Step 1: Freeze all layers except the last one (classification head)
for param in model.base_model.parameters():
    param.requires_grad = False

# If you'd like to fine-tune additional layers (e.g., the last 2 layers), you can unfreeze those layers as well
for param in model.base_model.encoder.layer[-2:].parameters():
    param.requires_grad = True

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


This loop freezes all the layers except for the final classification head. If you wish to fine-tune more than just the last layer, you can modify the loop to unfreeze the last two or three layers for retraining.

## Step 2: Set up fine-tuning with PEFT
Once you've identified the fine-tuning parameters, you can set up the process. For this example, we will use the Hugging Face Transformers library, which provides an easy interface for model fine-tuning. If you are unfamiliar with Hugging Face, it’s a popular open-source natural language processing (NLP) library that allows you to load pretrained models and fine-tune them for specific tasks.

If Hugging Face is new to you, here’s how it integrates into this process:
- Hugging Face provides pretrained models like BERT that are used in this example.
- We use their Trainer and TrainingArguments classes to handle the fine-tuning process, which allows us to specify parameters such as the number of epochs, batch size, and datasets to use.

In the code example below, you’ll see how we apply these concepts. Additionally, note that the actual PEFT process happens because the model’s layers were frozen in Step 1, so only the classification head (or additional layers, if specified) is fine-tuned here.

### Instructions for fine-tuning with PEFT
- Freeze the layers of the model (as shown in the previous code block).
- Set up the fine-tuning process using Hugging Face’s Trainer class and TrainingArguments, continuing from Step 1.
- Fine-tune the model based on the trainer setup, which is also shown in this code block.

In [12]:
from transformers import Trainer, TrainingArguments
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)


# Step 1: Set training arguments for fine-tuning the model
training_args = TrainingArguments(
    output_dir='./results',             # Directory where results will be stored
    num_train_epochs=3,                 # Number of epochs (full passes through the dataset)
    per_device_train_batch_size=16,     # Batch size per GPU/CPU during training
    #evaluation_strategy="epoch",        # Evaluate the model at the end of each epoch
)

# Step 2: Fine-tune only the final classification head (since earlier layers were frozen)
trainer = Trainer(
    model=model,                        # Pre-trained BERT model with frozen layers
    args=training_args,                 # Training arguments
    train_dataset=train_data,           # Training data for fine-tuning
    eval_dataset=val_data,              # Validation data to evaluate performance during training
)

# Step 3: Train the model using PEFT (this performs PEFT because layers were frozen in Step 1)
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyError: 1590

**Note:**

- The `Trainer` class from Hugging Face is responsible for setting up the fine-tuning process.
- The line `trainer.train()` fine-tunes the model with PEFT, leveraging the frozen layers from Step 1.
- The comment `# Fine-tune only the final classification head` has been revised to clarify that this is setting up the trainer for fine-tuning based on the frozen layers from Step 1.

## Step 3: Monitor and evaluate performance
After fine-tuning the model with PEFT, it is important to evaluate the model's performance and compare it to traditional fine-tuning methods. PEFT achieves similar or even better performance with less computational cost.

### Evaluation
Use standard evaluation metrics (e.g., accuracy, F1 score) to monitor the fine-tuned model's performance on the validation and test sets.

In [13]:
# Evaluate the model
results = trainer.evaluate(eval_dataset=test_data)
print(f"Test Accuracy: {results['eval_accuracy']}")

KeyError: 0

## Step 4: Optimize PEFT for your task
PEFT can be further optimized for specific tasks by experimenting with different sets of parameters or layers to fine-tune. You can also try adjusting the learning rate or batch size to see how they impact the model’s performance.

### Optimization ideas
- Fine-tune additional layers (e.g., the last two to three layers instead of just the final classification head).
- Adjust hyperparameters such as learning rate and number of epochs to find the best configuration for your task.

In [ ]:
# Example of adjusting learning rate for PEFT optimization
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=5e-5,  # Experiment with different learning rates
    num_train_epochs=5,
    per_device_train_batch_size=16,
)

## Conclusion
PEFT is an efficient method for fine-tuning large pretrained models, allowing you to save computational resources and time without sacrificing performance. By focusing on fine-tuning a subset of parameters, you can achieve task-specific improvements while keeping the rest of the model intact. This makes PEFT particularly useful when hardware resources are limited or rapid experimentation is needed.

We use BERT for sequence classification, with three output labels in this example. The training arguments are defined using the TrainingArguments class, where you can specify the batch size, number of epochs, and evaluation strategy. Trainer is used to handle the fine-tuning process by taking care of the training loop, backpropagation, and model updates.

## Step 3: Evaluate the model
After fine-tuning the model, it’s time to evaluate its performance on the test set. This allows you to measure how well the model generalizes to new, unseen data.

### Instructions
- Evaluate the fine-tuned model on the test set using standard metrics such as accuracy, F1 score, precision, and recall.
- Analyze the results to determine how well the model performs.